In [15]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import findspark
import os

os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'

findspark.init()
findspark.find()

'/usr/local/lib/python3.8/dist-packages/pyspark'

In [17]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Spark Caching") \
    .getOrCreate()

sc = SparkContext.getOrCreate(SparkConf())
sc.setCheckpointDir('/user/ivanrychko/analytics/test_checkpoints')

In [9]:
book = [('Harry Potter and the Goblet of Fire', 'J. K. Rowling', 322),
        ('Nineteen Eighty-Four', 'George Orwell', 382),
        ('Jane Eyre', 'Charlotte Brontë', 159),
        ('Catch-22', 'Joseph Heller', 174),
        ('The Catcher in the Rye', 'J. D. Salinger', 168),
        ('The Wind in the Willows', 'Kenneth Grahame', 259),
        ('The Mayor of Casterbridge', 'Thomas Hardy', 300),
        ('Bad Girls', 'Jacqueline Wilson', 299)
        ]

# данные второго датафрейма
library = [
    (322, "1"),
    (250, "2"),
    (400, "2"),
    (159, "1"),
    (382, "2"),
    (322, "1")
]

# названия атрибутов
columns = ['title', 'author', 'book_id']
columns_library = ['book_id', 'Library_id']

# создаём датафреймы
df = spark.createDataFrame(data=book, schema=columns)
df_library = spark.createDataFrame(data=library, schema=columns_library)

# делаем join
df_join = df.join(df_library, ['book_id'], 'leftanti').select('title')
df_cache = df_join.cache()

---

In [10]:
df_cache.show()

df_cache.explain()

+--------------------+
|               title|
+--------------------+
|           Bad Girls|
|            Catch-22|
|The Catcher in th...|
|The Mayor of Cast...|
|The Wind in the W...|
+--------------------+

== Physical Plan ==
InMemoryTableScan [title#394]
   +- InMemoryRelation [title#394], StorageLevel(disk, memory, deserialized, 1 replicas)
         +- *(5) Project [title#394]
            +- *(5) SortMergeJoin [book_id#396L], [book_id#400L], LeftAnti
               :- *(2) Sort [book_id#396L ASC NULLS FIRST], false, 0
               :  +- Exchange hashpartitioning(book_id#396L, 200), ENSURE_REQUIREMENTS, [plan_id=218]
               :     +- *(1) Project [title#394, book_id#396L]
               :        +- *(1) Scan ExistingRDD[title#394,author#395,book_id#396L]
               +- *(4) Sort [book_id#400L ASC NULLS FIRST], false, 0
                  +- Exchange hashpartitioning(book_id#400L, 200), ENSURE_REQUIREMENTS, [plan_id=227]
                     +- *(3) Project [book_id#400L]


In [18]:
df_cache.checkpoint()

DataFrame[title: string]

In [20]:
df_check = df_cache.checkpoint()
df_check.explain()

== Physical Plan ==
*(1) Scan ExistingRDD[title#394]


